In [1]:
from utils import getLoaderDataset

N = 256
B = 5

loader, dataset = getLoaderDataset(N, B, "./datasets/shakespear_corpus.txt")

In [2]:
print(len(dataset))

1115137


In [3]:
from model import GPT, train_model

L = 6
d = 384
d_ff = 4 * d
h = 6
V = dataset.get_vocab_size()

model = GPT(B, L, d, d_ff, N, h, V)

In [7]:
model, losses = train_model(model, loader, dataset)

batch 0, Loss : 4.304887771606445
batch 1, Loss : 3.9289886951446533
batch 2, Loss : 3.684896945953369
batch 3, Loss : 3.546851634979248
batch 4, Loss : 3.2864620685577393
batch 5, Loss : 3.3008835315704346
batch 6, Loss : 3.2625365257263184
batch 7, Loss : 3.3129429817199707
batch 8, Loss : 3.2393271923065186
batch 9, Loss : 3.1395046710968018
batch 10, Loss : 3.1759374141693115
batch 11, Loss : 3.0591042041778564
batch 12, Loss : 3.1147913932800293
batch 13, Loss : 3.0061724185943604
batch 14, Loss : 2.957833766937256
batch 15, Loss : 2.9745867252349854
batch 16, Loss : 2.9155819416046143
batch 17, Loss : 2.9046926498413086
batch 18, Loss : 2.9079174995422363
batch 19, Loss : 2.8551058769226074
batch 20, Loss : 2.902981758117676
batch 21, Loss : 2.7915217876434326
batch 22, Loss : 2.9033091068267822
batch 23, Loss : 2.872188091278076
batch 24, Loss : 2.851562023162842
batch 25, Loss : 2.705273151397705
batch 26, Loss : 2.8419995307922363
batch 27, Loss : 2.75630521774292
batch 28, Lo

KeyboardInterrupt: 

In [4]:
import torch

cpkt = torch.load("./runs/model_16700.pt", map_location=torch.device('cpu'))
model.load_state_dict(cpkt)
print(model)

GPT(
  (WTE): Embedding(65, 384)
  (WPE): WPE()
  (blocks): ModuleList(
    (0-5): 6 x Block(
      (CausalSelfAttn): QKVAttention()
      (W1): Linear(in_features=384, out_features=1536, bias=True)
      (W2): Linear(in_features=1536, out_features=384, bias=True)
      (LayerNorm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (LayerNorm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    )
  )
  (Dropout): Dropout(p=0.2, inplace=False)
  (Final_LayerNorm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  (LM_Head): Linear(in_features=384, out_features=65, bias=True)
)


In [8]:
from utils import generate

new_tokens = generate(model, dataset.encode("Let us kill him, and we'll have corn at our own price.!"), 500)
print(dataset.decode(new_tokens))

/home/gordon/Documents/edu/gordon_ms/Sem3/DeepLearning/Shakespeak/utils.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_c = torch.tensor(char_id).reshape(shape=(1, 1))


Let us kill him, and we'll have corn at our own price.! What then?

EXTON:
A sister, manage that he mood this in father, no,
teating hardon'd spitut!

POMPEY:
The sends he selfsion that doublet this time maid,
becauses an hour of Richard have speak should shop caught
intercliner'd to fixe yound, I there executor in unnto yand
to his cate, an aid evil, yexathing'd's walvicke, hid;
Whatell may nother the umourddger dour
An, lifthere mere, nour nagn,
Wher ine's, inghee? dd; munoun'd Than, whindrst,
St
Stoun meagooune? har.'st, y thinou! wakid iffffft
P
